In [1]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models


In [2]:
encoder_input_data = np.load('enc_in_data.npy')
decoder_input_data = np.load('./dec_in_data.npy')
decoder_output_data = np.load('./dec_tar_data.npy')


In [3]:
VOCAB_SIZE = 1894
encoder_inputs = tf.keras.layers.Input(shape=( None , ))

encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    378800      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    378800      input_2[0][0]                    
______________________________________________________________________________________________

In [4]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=75 ) 
model.save( 'model.h5' ) 

Train on 564 samples
Epoch 1/75
564/564 [==============================] - 10s 18ms/sample - loss: 1.2931
Epoch 2/75
564/564 [==============================] - 4s 7ms/sample - loss: 1.1177
Epoch 3/75
564/564 [==============================] - 4s 7ms/sample - loss: 1.0965
Epoch 4/75
564/564 [==============================] - 4s 7ms/sample - loss: 1.0772
Epoch 5/75
564/564 [==============================] - 4s 7ms/sample - loss: 1.0577
Epoch 6/75
564/564 [==============================] - 4s 8ms/sample - loss: 1.0414
Epoch 7/75
564/564 [==============================] - 4s 8ms/sample - loss: 1.0262
Epoch 8/75
564/564 [==============================] - 4s 7ms/sample - loss: 1.0132
Epoch 9/75
564/564 [==============================] - 5s 8ms/sample - loss: 0.9994
Epoch 10/75
564/564 [==============================] - 4s 7ms/sample - loss: 0.9869
Epoch 11/75
564/564 [==============================] - 4s 7ms/sample - loss: 0.9718
Epoch 12/75
564/564 [==============================] - 4s 8ms/

In [7]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [8]:

enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter question : Hello


NameError: name 'tokenizer' is not defined